# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file_to_load = "output_data/cities.csv"
cities_df = pd.read_csv(file_to_load)
print(cities_df.shape)
cities_df.head(3)

(580, 11)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed
0,Smirnykh,67,RU,1603261147,73,49.75,142.83,46.98,46.98,46.98,3.29
1,Hilo,1,US,1603261147,73,19.73,-155.09,71.60,69.01,75.20,3.36
2,New Norfolk,100,AU,1603261127,53,-42.78,147.06,59.20,57.99,61.00,1.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [6]:
# Using Lat & Lng as locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
locations.head(2)

,Lat,Lng
0,49.75,142.83
1,19.73,-155.09


In [7]:
#Defining Humudity as the weight
humidity = cities_df["Humidity"].astype(float)
humidity.head(2)

0    73.0
1    73.0
Name: Humidity, dtype: float64

In [13]:
# Creating a Heatmap layer
heat_fig = gmaps.figure(center=(46.0, -5.0),zoom_level=2) # 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)
heat_fig.add_layer(heat_layer)
heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Narrowing down cities by (i) max_temp<80 & min_temp>75; (ii) Wind Speed < 10; (iii) Cloudiness == 0

vacation_places = cities_df.loc[(cities_df["Min Temp"]>75)&(cities_df["Max Temp"]<80)
                                &(cities_df["Wind Speed"]<10)&(cities_df["Cloudiness"]==0)]
vacation_places = vacation_places.dropna()
print(vacation_places.shape)
vacation_places

(9, 11)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed
52,Maţāy,0,EG,1603261159,56,28.42,30.78,76.26,76.26,76.26,7.14
58,Cabo San Lucas,0,MX,1603260928,68,22.89,-109.91,79.00,79.00,79.00,9.89
101,Buraidah,0,SA,1603260987,29,26.33,43.98,75.20,75.20,75.20,2.24
108,Broken Hill,0,AU,1603261173,16,-31.95,141.43,78.80,78.80,78.80,9.17
179,Tessalit,0,ML,1603261190,18,20.20,1.01,76.86,76.86,76.86,8.21
217,Lukulu,0,ZM,1603261199,19,-14.37,23.24,77.43,77.43,77.43,7.11
291,Adrar,0,MR,1603261218,20,20.50,-10.07,79.43,79.43,79.43,5.06
329,Safaga,0,EG,1603261228,61,26.73,33.94,77.00,77.00,77.00,6.93
498,Mulegé,0,MX,1603261274,55,26.88,-111.98,79.18,79.18,79.18,2.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
vaca_locations = vacation_places[["Lat", "Lng"]].astype(float)
print(vaca_locations.shape)
vaca_locations.head(2)

(9, 2)


,Lat,Lng
52,28.42,30.78
58,22.89,-109.91


In [17]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [18]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(vaca_locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [19]:
vacation_places["Hotel Name"] =""
vacation_places["Hotel Address"] =""
vacation_places["Hotel Rating"] =""
vacation_places = vacation_places.reset_index()
print(vacation_places.shape)
vacation_places.head(3)

#adjust in the hmw only includes hotel name

(9, 15)


,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed,Hotel Name,Hotel Address,Hotel Rating
0,52,Maţāy,0,EG,1603261159,56,28.42,30.78,76.26,76.26,76.26,7.14,,,
1,58,Cabo San Lucas,0,MX,1603260928,68,22.89,-109.91,79.00,79.00,79.00,9.89,,,
2,101,Buraidah,0,SA,1603260987,29,26.33,43.98,75.20,75.20,75.20,2.24,,,


In [28]:
def getHotel(lat,lng):
    params = {
        "keyword": "Hotel",
        "types": "lodging",
        "radius": 50000,
        "key": g_key,
        "location": f"{lat}, {lng}"
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
    pprint(response)    
    return response


In [29]:
results = []

for i, row in vacation_places.iterrows():
    function_response = getHotel(row["Lat"], row["Lng"])
    results.append(function_response)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 28.1138559, 'lng': 30.7504432},
                           'viewport': {'northeast': {'lat': 28.11517107989273,
                                                      'lng': 30.75170557989273},
                                        'southwest': {'lat': 28.11247142010728,
                                                      'lng': 30.74900592010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Grand Aton Hotel',
              'photos': [{'height': 3024,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/113426147043540488402">Zakaria '
                                                'Nabil</a>'],
                          'photo_reference': 'CmRaAAAAtJrT_lXJ2ay3D0K3rcmg967-3DkEAwWUlEkJ4PI-lTdG

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAHgxtM5eL7RYJxmS93kQrBkGBMyHCAiyBEhHDJI0E5Lt_RNdjOLP9gtCLZgpLPbBqleknQd1NKtxTWNOn_HxK9VbngsRQwSyTNznBmlXE08ahG_tMvDmyObUHLoshj4KD7fjzR2Dvi61uMR1GWtFVLsDrKaDHLP7QLOCtze0Ubz-yIpj_orO1b_HeskwgRbx1z-H7xc8d4vSvpnCDkUODmPBm4DUDitUREGOaA5zWcyRH6S899_vOtwYLutQNyWGE5sn-VEMjW0xNG4ZIRSjmWmkhGrqcl7ZWRmd4M9OAvhiBD1fqqa7F_WQ81sJYq9mafvu7IQJM72EAVsFUG_PtVGRYmramQAN7V_vN4EnGoOsTDrFusT3yEPlxbCqFWvKXdhZahfMbmnVWAIdBZQUjjcSEPvnBUNaoOYtp4cILx3FHV4aFHS4RRTK3cuKD_fgbnMAmKz6z3t6',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 22.8748106, 'lng': -109.912136},
                           'viewport': {'northeast': {'lat': 22.87649042989272,
                                                      'lng': -109.9113781201073},
                                        'southwest': {'lat': 22.87379077010728,
                                                      'lng': -109.9140777798927}}},
              'icon': 'https

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAGwsrMF9z43cl0sJk5px0axGc3ww5aKdh9ElO_4ziW4zgLCV4l2T5TOC4NCPiyPYG1YsCVPPbkOSj7zOugvR-RkZmRg9YABxk9BbFWKD_bSrFoaqKQngWSDhUzwUFgPlF16G4dY5OtmBeJYXzmmi0k150X9CO33cuwZWC1B4_G5rp_BZ_kSMmOij_lMS7B10NCrDJf0HNEmKqG8qelWca8yM8AUo896oI5XQbFl9T1Roxc50pxXW3Pk1ANJNSy1ru6qOT4YZ8pAKtuGxHWckDntv1rTFm62m9qnvwgZRrv3Bj2e8tOAK9yTslaFcyRN_wqOvgYSBA7fs2flprNt0fFOuc-JyYR2ZNVVYywJZLoVtzhLBDLSSkdl3d3ZSDFZx4a2Q0Lcg7vJt_gX085Vyf-gSEMXnsHlkdf5kB2qeTRQ1Kf4aFJ_H23I0P9cH03QixFIfAZ25mwex',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 26.0392885, 'lng': 43.9580516},
                           'viewport': {'northeast': {'lat': 26.04063832989272,
                                                      'lng': 43.95940142989271},
                                        'southwest': {'lat': 26.03793867010728,
                                                      'lng': 43.95670177010727}}},
              'icon': 'https://

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAD0jgG8wyrjM8FZgG1jVsBzhbW7EJetLOLNs-97uUNjIKejhWGL6_ndE0UNiRLII81SuxMeQgfCPGg-3Q6l86ljd9xpT2VKoLtAHFMYCv1ExpT-o5vV_hWV5DNNQY0WudEESFzn_BxhMpitjQF6_Cpm5UXPuweB7JPnKKALBGxR1un9WRsZxhvQWmWGBLCDCS10A_mSyZe8_fU5SWvCGcg3_3xQC9gKDzZjKxHJbmmYObt9Uof-7bvZ08EPgmptWx-98A8mkNBjMuKjQL4kiN6r_tU7LCL48uBjTg1tBtKh2_whPg2gNw0bkvQnlfNnSTap1Dn3a3URr7OWA3sOLU9UoYS0rMf88AdWjUsmH1-A30-rUzd6As8Gu_ADI9F4dA5u3AiorVuSCO7yb1F5tE5wSEMpacsPaqN0flCl8tcPcgVIaFFntNpobi0dco432RGrXdUjrsvuf',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -31.89595719999999,
                                        'lng': 141.5981271},
                           'viewport': {'northeast': {'lat': -31.89509867010727,
                                                      'lng': 141.5994562298928},
                                        'southwest': {'lat': -31.89779832989272,
                                                      'lng': 14

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -14.3912737, 'lng': 23.2463243},
                           'viewport': {'northeast': {'lat': -14.38983322010728,
                                                      'lng': 23.24777112989272},
                                        'southwest': {'lat': -14.39253287989272,
                                                      'lng': 23.24507147010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Samwanji Guest Lodge',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1080,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/106831056786821066912">Musenga '
                                   

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 27.3291401, 'lng': -112.2557258},
                           'viewport': {'northeast': {'lat': 27.33031542989273,
                                                      'lng': -112.2544629201073},
                                        'southwest': {'lat': 27.32761577010728,
                                                      'lng': -112.2571625798927}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Hotel Las Casitas Santa Rosalia',
              'photos': [{'height': 1960,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/109747078074639290521">VICTOR '
                                                'CEBALLOS</a>'],
                          'photo_reference': 'CmRaAAAAxUV7OH56MYZuf40XKBcE4mA

In [22]:
print(len(results))
print(results[0]["results"][0]["name"])
print(results[0]["results"][0]["vicinity"])
print(results[0]["results"][0]["rating"])


9
Queen Isis Floating Hotel
Al Kady Dock, Qism Minya, Al Minya
3.5


In [23]:
for i, row in vacation_places.iterrows():
    try:
        vacation_places.loc[i,"Hotel Name"] = results[i]["results"][0]["name"]
        vacation_places.loc[i, "Hotel Address"] = results[i]["results"][0]["vicinity"]
        vacation_places.loc[i, "Hotel Rating"]= results[i]["results"][0]["rating"]
    except:
        print(f"Missing result for {row['City']}")

Missing result for Tessalit
Missing result for Adrar


In [24]:
vacation_places

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Min Temp,Max Temp,Wind Speed,Hotel Name,Hotel Address,Hotel Rating
0,52,Maţāy,0,EG,1603261159,56,28.42,30.78,76.26,76.26,76.26,7.14,Queen Isis Floating Hotel,"Al Kady Dock, Qism Minya, Al Minya",3.5
1,58,Cabo San Lucas,0,MX,1603260928,68,22.89,-109.91,79.00,79.00,79.00,9.89,Waldorf Astoria Los Cabos Pedregal,"Camino del Mar 1, Pedregal, Cabo San Lucas",4.8
2,101,Buraidah,0,SA,1603260987,29,26.33,43.98,75.20,75.20,75.20,2.24,Swiss International Resort Unaizah - Al Qassim,Al Hajeb، Unayzah,4
3,108,Broken Hill,0,AU,1603261173,16,-31.95,141.43,78.80,78.80,78.80,9.17,Broken Hill Outback Resort,"Barrier Hwy, Broken Hill",4.4
4,179,Tessalit,0,ML,1603261190,18,20.20,1.01,76.86,76.86,76.86,8.21,,,
5,217,Lukulu,0,ZM,1603261199,19,-14.37,23.24,77.43,77.43,77.43,7.11,Samwanji Guest Lodge,D557,3.2
6,291,Adrar,0,MR,1603261218,20,20.50,-10.07,79.43,79.43,79.43,5.06,,,
7,329,Safaga,0,EG,1603261228,61,26.73,33.94,77.00,77.00,77.00,6.93,Steigenberger Makadi,Madinat Makadi Golf Road Makadi Bay-Hurghada-S...,4.8
8,498,Mulegé,0,MX,1603261274,55,26.88,-111.98,79.18,79.18,79.18,2.66,Hotel Las Casitas Santa Rosalia,"México 1 Km 1.5, Costa Azul, Santa Rosalía",4.4


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_places.iterrows()]
locations = vaca_locations[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
heat_fig.add_layer(hotel_markers)
heat_fig

# Display Map


#marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)
#fig = gmaps.figure()
#fig.add_layer(marker_layer)
#fig


Figure(layout=FigureLayout(height='420px'))